In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#basePath = "/kaggle/input/rsna-intracranial-hemorrhage-detection" 
#os.chdir(basePath)
print(os.getcwd())
TRAIN_DIR = '/kaggle/input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/'
TEST_DIR = '/kaggle/input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/'

print("Number of Training Images:", len(os.listdir(TRAIN_DIR)))
print("Number of Testing Images:", len(os.listdir(TEST_DIR)))

# Any results you write to the current directory are saved as output.

In [ ]:
import pydicom as pdc # Opens up dicom files to read
import matplotlib.pyplot as plt

# Create Handler for reading dicom files
dcmHandler = pdc.dcmread(TRAIN_DIR+"ID_000039fa0.dcm")

# Various info regarding image
print(dcmHandler)

# Show pixel image size
print("Pixel Image Size: ", dcmHandler.pixel_array.shape)

# Show intensity Array
print("\nIntensity Array: ")
print(dcmHandler.pixel_array)

# Show image (Can be changed by changing cmap = plt.cm.jet)
plt.imshow(dcmHandler.pixel_array, cmap="gray")

In [ ]:
# Obtain list of Patient ID's
TRAIN_ID = []
TRAIN_ID_RAW = os.listdir(TRAIN_DIR)
for ID in TRAIN_ID_RAW:
    TRAIN_ID.append(ID.split(".")[0])


In [ ]:
# Open Training CSV to obtain outputs for each patient ID
TrainCsvHdlr = pd.read_csv("/kaggle/input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv")
TrainCsvHdlr.iloc[1]#['ID']


In [ ]:
import pprint # Just to make console printing formatted nicer
from collections import OrderedDict # Used for implementing ordered dictionaries

outputs = OrderedDict()

# I picked 10000 because it took too long for it to process everything. 
for index in range(0, 6000):
        patientID = str(TrainCsvHdlr.iloc[index]['ID']).split("_")[1]
        if patientID not in  outputs:
            outputs[patientID] = []
        outputs[patientID].append(TrainCsvHdlr.iloc[index]["Label"])


In [ ]:
'ID_63eb1e259.dcm' in os.listdir(TRAIN_DIR)
TRAIN50 = os.listdir(TRAIN_DIR)[0:99]

Xtrain = []
Ytrain = []
for key, value in outputs.items():
    #print(pdc.dcmread(TRAIN_DIR+"ID_"+key+".dcm").pixel_array, value)
    Xtrain.append(pdc.dcmread(TRAIN_DIR+"ID_"+key+".dcm").pixel_array)
    Ytrain.append(value)
print((np.array(Xtrain)).shape)  
print((np.array(Ytrain)).shape)    
    


In [ ]:
#Start NN
import tensorflow as tf
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(512,512)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(6 , activation=tf.nn.softmax)
])
print(np.array(Ytrain).shape)
model.compile(optimizer='rmsprop',
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.fit(np.array(Xtrain), np.array(Ytrain), epochs=10)

In [ ]:
dcmHandler = pdc.dcmread(TRAIN_DIR+"ID_63eb1e259.dcm")
x=[]
x.append(dcmHandler.pixel_array)
dcmHandler = pdc.dcmread(TRAIN_DIR+"ID_2669954a7.dcm")
x.append(dcmHandler.pixel_array)

print(outputs['63eb1e259'])
predictions = model.predict(np.array(x))
predictions